In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import os

os.getcwd()

'/Users/tomokitakata/Desktop/research/dev/muldistribution/models'

# データ読み込みと前処理

In [2]:
# 医療機関POIデータ読み込み
rihabili = gpd.read_file('../data/medicalpoi/2010/リハビリテーション科.shp')
gastro = gpd.read_file('../data/medicalpoi/2010/消化器内科.shp')
cardio = gpd.read_file('../data/medicalpoi/2010/循環器内科.shp')
uro = gpd.read_file('../data/medicalpoi/2010/泌尿器内科.shp')
# gridデータ読み込み
grid = gpd.read_file('../data/grid/wholejapan_grid/wholejapan_grid.shp')
# 医療機関POIデータが各gridに何件あるかを集計
# gpkgデータとして出力

In [3]:
import geopandas as gpd
from pathlib import Path

# ── 0. パス設定 ─────────────────────────────────
poi_files = {
    "Rehabilitation": "../data/medicalpoi/2010/リハビリテーション科.shp",
    "Gastro":         "../data/medicalpoi/2010/消化器内科.shp",
    "Cardio":         "../data/medicalpoi/2010/循環器内科.shp",
    "Urology":        "../data/medicalpoi/2010/泌尿器内科.shp",
}
grid_path = "../data/grid/wholejapan_grid/wholejapan_grid.shp"
out_gpkg  = Path("grid_with_poi_counts.gpkg")

# ── 1. グリッド読み込み（基準 CRS を決める） ──────────
grid = gpd.read_file(grid_path)
grid = grid.set_crs(4326) if grid.crs is None else grid   # ↓ポイント類もこの座標系に合わせる

# ユニークキー列（無ければ index を使う）
gid_col = "grid_id"
if gid_col not in grid.columns:
    grid[gid_col] = grid.index.astype(int)

# ── 2. 各診療科について「グリッド内件数」を計算 ───────
for eng, path in poi_files.items():
    poi = gpd.read_file(path)
    poi = poi.set_crs(4326) if poi.crs is None else poi     # 度数系と仮定
    poi = poi.to_crs(grid.crs)                              # CRS 揃える

    # a) 空間結合（各ポイントが属するグリッド ID を取る）
    joined = gpd.sjoin(poi[[poi.geometry.name]],
                       grid[[gid_col, grid.geometry.name]],
                       predicate="within",
                       how="left")

    # b) グリッド単位で件数カウント
    cnt = (
        joined
        .groupby(gid_col)
        .size()
        .rename(f"{eng}_cnt")        # 列名例: Rehabilitation_cnt
        .to_frame()
    )

    # c) グリッド本体にマージ（無いところは 0）
    grid = grid.merge(cnt, on=gid_col, how="left")
    grid[f"{eng}_cnt"] = grid[f"{eng}_cnt"].fillna(0).astype(int)

# ── 3. GeoPackage に書き出し  (全診療科まとめて 1 レイヤ) ─
# out_gpkg.unlink(missing_ok=True)        # 既存を削除
# grid.to_file(out_gpkg, layer="grid_poi_counts", driver="GPKG")
# print("✅ 書き出し完了 →", out_gpkg.resolve())

In [4]:
"""
make_ml_table.py
────────────────────────────────────────────────────────
1. 日本全国メッシュ (wholejapan_grid.shp) を基盤に
2. 各診療科 POI 件数を集計           → *_cnt 列
3. 人口メッシュ (pop_grid.gpkg) を合体 → population 列
4. 機械学習用の DataFrame (= geometry なし) を保存
   ・grid_id, centroid_X/Y, 各 *_cnt, population
────────────────────────────────────────────────────────
"""

from pathlib import Path
import geopandas as gpd
import pandas as pd

# ─── 入出力パス ─────────────────────────────────────────
grid_path = Path("../data/grid/wholejapan_grid/wholejapan_grid.shp")
pop_path  = Path("../data/pop/2010/pop_grid.gpkg")           # ★人口
poi_files = {
    "Rehabilitation": "../data/medicalpoi/2010/リハビリテーション科.shp",
    "Gastro":         "../data/medicalpoi/2010/消化器内科.shp",
    "Cardio":         "../data/medicalpoi/2010/循環器内科.shp",
    "Urology":        "../data/medicalpoi/2010/泌尿器内科.shp",
}

out_geo = Path("grid_with_features.gpkg")   # 中間 GeoPackage
out_df  = Path("ml_table.parquet")          # 最終 ML テーブル

pop_col = "Total population"               # 人口列名 in pop_grid.gpkg
gid_col = "grid_id"                        # → なければ index で作成

# ─── 1. 基盤グリッド読み込み ────────────────────────────

# 変更後（存在すればそのまま、無ければ 4326 をセット）
grid = gpd.read_file(grid_path)
grid = grid if grid.crs is not None else grid.set_crs(4326)
if gid_col not in grid.columns:
    grid[gid_col] = grid.index.astype(int)

# ─── 2. 各診療科の POI → グリッド件数集計 ───────────────
for name, path in poi_files.items():
    poi = gpd.read_file(path).set_crs(4326).to_crs(grid.crs)
    join = gpd.sjoin(poi[[poi.geometry.name]],
                     grid[[gid_col, grid.geometry.name]],
                     predicate="within", how="left")
    cnt  = (
        join.groupby(gid_col).size()
             .rename(f"{name}_cnt")
             .to_frame()
    )
    grid = grid.merge(cnt, on=gid_col, how="left")
    grid[f"{name}_cnt"] = grid[f"{name}_cnt"].fillna(0).astype("uint16")

# ─── 3. 人口メッシュの結合 ──────────────────────────────
pop = gpd.read_file(pop_path)            # 読み込み
if pop.crs is None:                      # ← CRS が無い時だけラベル付与
    pop = pop.set_crs(4326)
pop = pop.to_crs(grid.crs)               
if pop_col not in pop.columns:
    raise KeyError(f"列 {pop_col} が見当たりません")

# → geometry 同士で重ね合わせ（intersects で十分）
pop_trim = pop[[pop_col, pop.geometry.name]]
join_pop = gpd.sjoin(pop_trim, grid[[gid_col, grid.geometry.name]],
                     predicate="intersects", how="left")

pop_sum = (
    join_pop.groupby(gid_col)[pop_col].sum()
            .rename("population")
            .to_frame()
)

grid = grid.merge(pop_sum, on=gid_col, how="left")
grid["population"] = grid["population"].fillna(0).astype("uint32")

# ─── 4. セントロイド座標列を追加 (ML 用) ────────────────
# 投影系 (例: EPSG:6697 = JGD2011 / Albers) へ一時変換して計算
proj_epsg = 6697
grid_metric = grid.to_crs(proj_epsg)
cent        = grid_metric.geometry.centroid   # ←メートル系なら警告無し
grid["centroid_X"] = cent.x
grid["centroid_Y"] = cent.y

# ─── 5. 保存 ───────────────────────────────────────────
#   a) GeoPackage (QGIS で確認用)
# out_geo.unlink(missing_ok=True)
# grid.to_file(out_geo, layer="grid_features", driver="GPKG")
# print("✅ GeoPackage:", out_geo.resolve())

#   b) 機械学習テーブル (geometry を除く)
df = grid.drop(columns="geometry")
# df.to_csv("ml_table.csv", index=False)

/var/folders/nw/cvnq7pwd21qdy7ff97bszbq40000gn/T/ipykernel_9511/204052548.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent        = grid_metric.geometry.centroid   # ←メートル系なら警告無し


In [12]:
df.head()

,KEY_CODE,MESH1_ID,MESH2_ID,MESH3_ID,OBJ_ID,grid_id,Rehabilitation_cnt,Gastro_cnt,Cardio_cnt,Urology_cnt,population,centroid_X,centroid_Y
0,66440000,6644,00,00,1.0,0,0,0,0,0,4,144.00625,44.004167
1,66440001,6644,00,01,2.0,1,0,0,0,0,0,144.01875,44.004167
2,66440002,6644,00,02,3.0,2,0,0,0,0,0,144.03125,44.004167
3,66440003,6644,00,03,4.0,3,0,0,0,0,0,144.04375,44.004167
4,66440004,6644,00,04,5.0,4,0,0,0,0,0,144.05625,44.004167


# random forest の構築

In [7]:
# rf_gridsearch_by_section.py
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from joblib import parallel_backend  
from tqdm  import tqdm

# ── 0.  入力ファイル ────────────────────────────────
df_path = Path("ml_table.csv")          # 作成済み CSV
df = pd.read_csv(df_path)

# ── 1.  共通設定 ──────────────────────────────────
features = ["population", "centroid_X", "centroid_Y"]
targets  = {
    "Rehabilitation_cnt": "Rehabilitation",
    "Gastro_cnt":         "Gastroenterology",
    "Cardio_cnt":         "Cardiology",
    "Urology_cnt":        "Urology (Med.)"
}
test_size    = 0.2
random_state = 42
cv_folds     = 5

# RMSE（正の値）を返す scorer
rmse_scorer = make_scorer(
    lambda y, yhat: np.sqrt(mean_squared_error(y, yhat)),
    greater_is_better=False       # GridSearch は「大きいほど良い」を期待
)

# 代表的なハイパーパラメータセット
param_grid = {
    "n_estimators": [20, 50],        # ←例：とりあえず2通り
    "max_depth": [10, 15],
    "max_features": ["sqrt"],
    "min_samples_leaf": [1, 2],
    "min_samples_split": [2, 5],
}

print(f"─ Grid search: {len(param_grid['n_estimators'])*len(param_grid['max_depth'])*len(param_grid['min_samples_split'])*len(param_grid['min_samples_leaf'])*len(param_grid['max_features'])} parameter combos × {cv_folds}-fold CV\n")

# ── 2.  診療科ごとに学習 ───────────────────────────
for col, label in tqdm(targets.items()):
    X = df[features]
    y = df[col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    base_rf = RandomForestRegressor(random_state=random_state)

    gsearch = GridSearchCV(
        base_rf,
        param_grid,
        scoring={"RMSE": rmse_scorer, "R2": "r2"},
        refit="RMSE",         # = 最小 RMSE を選ぶ
        cv=cv_folds,
        n_jobs=-1,            # CPU 全コア
        verbose=0
    )

    # scikit‑learn 1.4 以降は loky backend がデフォルト
    with parallel_backend("loky"):
        gsearch.fit(X_train, y_train)

    best = gsearch.best_estimator_
    y_pred = best.predict(X_test)

    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test   = r2_score(y_test, y_pred)

    print(f"{label:20s} |  RMSE_test = {rmse_test:7.3f}   R²_test = {r2_test:6.3f}")
    print("   best_params :", gsearch.best_params_, "\n")

─ Grid search: 16 parameter combos × 5-fold CV



 25%|██▌       | 1/4 [05:17<15:53, 317.68s/it]

Rehabilitation       |  RMSE_test =   0.653   R²_test =  0.452
   best_params : {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50} 



 50%|█████     | 2/4 [10:19<10:16, 308.22s/it]

Gastroenterology     |  RMSE_test =   0.604   R²_test =  0.493
   best_params : {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50} 



 75%|███████▌  | 3/4 [15:26<05:07, 307.69s/it]

Cardiology           |  RMSE_test =   0.436   R²_test =  0.412
   best_params : {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50} 



100%|██████████| 4/4 [20:22<00:00, 305.56s/it]

Urology (Med.)       |  RMSE_test =   0.552   R²_test =  0.247
   best_params : {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50} 

